<a href="https://colab.research.google.com/github/clauabad/TP-A52-ApprentissageSupervise/blob/main/Magasin_application%2BBellman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Application des équations de Bellman sur un projet réel**

##**1. Imports **

In [29]:
import os
import sys

sys.path.insert(1, "/content/ApplicationRL")

In [30]:
!pip -q install meshio

##**2. Découverte de l'environnement de travail**

In [31]:
from magasin import Magasin
from utils import bgr8_to_jpeg
import ipywidgets
import numpy as np

from IPython.display import display

#####**2.1 Instanciation de la classe Magasin**

In [32]:
Magasin = Magasin("ApplicationRL/maillage_carte.msh","ApplicationRL/image_magasin.png")

In [33]:
Magasin.RECOMPENSE = 1
Magasin.RECOMPENSE_MUR = -10
Magasin.RECOMPENSE_NON_CIBLE = 0
Magasin.PROBA_MAX = 0.98

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


#####**2.2 Saisie de l'objectif**

In [34]:
def AfficheSelectionImage(change):
    numero = change['new']
    Magasin.ETAT_CIBLE = numero
    widget_image.value = bgr8_to_jpeg(Magasin.AfficheObjectifSurImage())

In [35]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin),width=640)
slider_numero = ipywidgets.IntSlider(value=80,min=0,max=Magasin.nombre_etats-1)

# Création du lien avec le slider
slider_numero.observe(AfficheSelectionImage, names='value')

# Création de l'interface d'acquisition
widget_presentation = ipywidgets.VBox([widget_image, slider_numero])

display(widget_presentation)

#####**2.3 Format de la table de transition**

In [36]:
Magasin.df_table_transition

,Etat,Action,Etat suivant,Proba,Récompense
0,0,0,225,0.98,0.0
1,0,0,227,0.01,0.0
2,0,0,226,0.01,0.0
3,0,1,225,0.01,0.0
4,0,1,227,0.98,0.0
...,...,...,...,...,...
2713,301,1,286,0.98,0.0
2714,301,1,301,0.01,-10.0
2715,301,2,289,0.01,0.0
2716,301,2,286,0.01,0.0


Le format de la variable table_transitions est le suivant :

- (nbr_etats, nbr_actions, nbr_etat_suivant,['idex_etat_suivant','proba_transition','recompense'])
= (nbr_etats, 3, 3,[1,1,1])

Regardons maintenant vers quels états nous dirige une action particulière sur un état donné :

In [37]:
etat = 301
action = 2

Magasin.table_transitions[etat,action,:]['index_etat_suivant']

array([289, 286, 301], dtype=int32)

Regardons les probabilités associées à cette action :

In [38]:
Magasin.table_transitions[etat,action,:]['proba_transition']

array([0.01, 0.01, 0.98], dtype=float32)

Puis les récompenses obtenues :


In [39]:
Magasin.table_transitions[etat,action,:]['recompense']

array([  0.,   0., -10.], dtype=float32)

#####**2.4 Format de la table des valeurs d'action**

In [40]:
Magasin.df_Qtable

,Etat courant,Action,Valeur
0,0,0,0.0
1,0,1,0.0
2,0,2,0.0
3,1,0,0.0
4,1,1,0.0
...,...,...,...
901,300,1,0.0
902,300,2,0.0
903,301,0,0.0
904,301,1,0.0


Le format de la variable table_valeur est le suivant : (nbr_etats, nbr_actions)

In [41]:
Magasin.Q_table[etat,:]

array([0., 0., 0.], dtype=float32)

#####**2.5 Format de la table des valeurs d'état**

In [42]:
Magasin.df_Vtable

,Etat courant,Valeur
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
297,297,0.0
298,298,0.0
299,299,0.0
300,300,0.0


Le format de la variable Vtable est le suivant : (nbr_etats,)

In [43]:
Magasin.V_table[etat]

np.float32(0.0)

##**3. Application des équations de Bellman**

On commence par initialiser l'environnement :

In [44]:
Magasin.RECOMPENSE = 1
Magasin.RECOMPENSE_MUR = -20
Magasin.RECOMPENSE_NON_CIBLE = -0.1
Magasin.PROBA_MAX = 0.9

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


Applique les équations de Bellman:

- Valeur des états :

$\quad\quad\quad{V_\pi }\left( s \right)={E_\pi }\left[ {{G_t}|{S_t} = s} \right]$

$\quad\quad\quad\quad\quad\quad= \sum\limits_a {\pi \left( {a|s} \right)\sum\limits_{s'} {\sum\limits_r {p\left( {s',r|s,a} \right)} } } \left[ {r + \gamma {V_\pi }\left( {s'} \right)} \right]$
<br>
<br>
- Valeur des actions :

$\quad\quad\quad{Q_\pi }\left( {s,a} \right) = {E_\pi }\left[ {{G_t}|{S_t} = s,{A_t} = a} \right]$

$\quad\quad\quad\quad\quad\quad\quad=\sum\limits_{s',r} {p\left( {s',r|s,a} \right)} \left[ {r + \gamma {V_\pi }\left( {s'} \right)} \right]$$

In [45]:
import numpy as np

episodes = 5
gamma = 0.5

for i in range(episodes):
  # Équation de Bellman - Fonction valeur d'actions
  for etat in range(0,Magasin.nombre_etats):
      for action in range(0,3):
          for etat_suivant in range(0,3):
              # Récupère la probabilité et la récompense
              proba = Magasin.table_transitions[etat,action,etat_suivant]['proba_transition']
              recompense = Magasin.table_transitions[etat,action,etat_suivant]['recompense']

              # Fonction valeur d'actions
              Magasin.Q_table[etat,action] = Magasin.Q_table[etat,action] + proba*(recompense + gamma*Magasin.V_table[etat_suivant])

  # Mise à jour de la table des valeurs d'état
  # avec une politique Greedy (pi(a|s) = 1 avec a=action greedy)
  for etat in range(0,Magasin.nombre_etats):
      index_max = np.where(Magasin.Q_table[etat,:] == np.amax(Magasin.Q_table[etat,:]))[0]
      Magasin.V_table[etat] = Magasin.Q_table[etat,np.random.choice(index_max)]

In [46]:
Magasin.Getdf_Qtable()

,Etat courant,Action,Valeur
0,0,0,-1.318750
1,0,1,-1.318750
2,0,2,-1.318750
3,1,0,-1.318750
4,1,1,-1.318750
...,...,...,...
901,300,1,-6.293750
902,300,2,-90.868752
903,301,0,-6.293750
904,301,1,-6.293750


In [47]:
Magasin.Getdf_Vtable()

,Etat courant,Valeur
0,0,-1.31875
1,1,-1.31875
2,2,-1.31875
3,3,-1.31875
4,4,-6.29375
...,...,...
297,297,-6.29375
298,298,-6.29375
299,299,-6.29375
300,300,-6.29375


#####**Affichage de la carte des valeurs d'états**

In [48]:
# Création de la colorMap
Magasin.CreationColorMap()

# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_Vtable),width=640)

display(widget_image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

#####**Affichage du parcours**

In [49]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin_objectif),width=640)
widget_iteration = ipywidgets.Text(value="0",description="Itération: ")
widget_etat = ipywidgets.Text(value="0",description="État: ")

widget_presentation = ipywidgets.VBox([widget_iteration,widget_etat,widget_image])

In [50]:
import time

objectif_atteint = False
max_iteration = 500
freq_mise_a_jour = 10

Magasin.InitImageTrajectoire()

iteration = 0
etat_courant = Magasin._ETAT_DEPART

display(widget_presentation)

time.sleep(1)

while objectif_atteint is False and iteration <= max_iteration:
    widget_etat.value = str(etat_courant)

    # Récupère la table des valeurs d'action de l'état en cours
    Q_ = Magasin.Q_table[etat_courant,:]

    # Récupère l'action optimale
    # si plusieurs actions sont optimales, alors tire une au hasard
    action = np.random.choice(np.where(Q_ == np.max(Q_))[0])

    # Simule l'action
    etat_courant, image_ = Magasin.SimuleAction(etat_courant,action)

    if etat_courant == Magasin.ETAT_CIBLE:
        objectif_atteint = True

    # Affiche la trajectoire
    if iteration%freq_mise_a_jour == 0:
      widget_image.value = bgr8_to_jpeg(image_)
      widget_iteration.value = str(iteration+1)

    iteration = iteration + 1
    time.sleep(0.1)


##**4. Politique Epsilon-Greedy**

In [51]:
Magasin.RECOMPENSE = 1
Magasin.RECOMPENSE_MUR = -20
Magasin.RECOMPENSE_NON_CIBLE = -0.1
Magasin.PROBA_MAX = 0.9

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


Applique les équations de Bellman:

- Valeur des états :

$\quad\quad\quad{V_\pi }\left( s \right)={E_\pi }\left[ {{G_t}|{S_t} = s} \right]$

$\quad\quad\quad\quad\quad\quad= \sum\limits_a {\pi \left( {a|s} \right)\sum\limits_{s'} {\sum\limits_r {p\left( {s',r|s,a} \right)} } } \left[ {r + \gamma {V_\pi }\left( {s'} \right)} \right]$
<br>
<br>
- Valeur des actions :

$\quad\quad\quad{Q_\pi }\left( {s,a} \right) = {E_\pi }\left[ {{G_t}|{S_t} = s,{A_t} = a} \right]$

$\quad\quad\quad\quad\quad\quad\quad=\sum\limits_{s',r} {p\left( {s',r|s,a} \right)} \left[ {r + \gamma {V_\pi }\left( {s'} \right)} \right]$$

In [52]:
import numpy as np
import sys
from scipy.special import softmax

Epsilon = 0.5
gamma = 0.5
episodes = 5

for i in range(episodes):
  # Équation de Bellman - Fonction valeur d'actions
  for etat in range(0,Magasin.nombre_etats):
      for action in range(0,3):
          for etat_suivant in range(0,3):
              # Récupère la probabilité et la récompense
              proba = Magasin.table_transitions[etat,action,etat_suivant]['proba_transition']
              recompense = Magasin.table_transitions[etat,action,etat_suivant]['recompense']

              # Equation de Bellman
              Magasin.Q_table[etat,action] = Magasin.Q_table[etat,action] + proba*(recompense + gamma*Magasin.V_table[etat_suivant])

  # Mise à jour de la table des valeurs d'état
  # avec une politique Greedy
  for etat in range(0,Magasin.nombre_etats):
      # Calcul de pi(a|s) en suivant une stratégie Epsilon-Greedy
      rng = np.random.default_rng()
      val = rng.uniform()
      if val > Epsilon:
          pi_ = [1/3, 1/3, 1/3]
          for action in range(0, 3):
              # Equation de Bellman
              Magasin.V_table[etat] = Magasin.V_table[etat] + pi_[action] * Magasin.Q_table[etat, action]
      else:
          index_max = np.where(Magasin.Q_table[etat, :] == np.amax(Magasin.Q_table[etat, :]))[0]
          Magasin.V_table[etat] = Magasin.Q_table[etat, np.random.choice(index_max)]

In [53]:
# Création de la colorMap
Magasin.CreationColorMap()

# Création des widgets
widget_image_colorMap = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_Vtable),width=640)

display(widget_image_colorMap)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [54]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin_objectif),width=640)
widget_iteration = ipywidgets.Text(value="0",description="Itération: ")
widget_etat = ipywidgets.Text(value="0",description="État: ")

widget_presentation = ipywidgets.VBox([widget_iteration,widget_etat,widget_image])

In [55]:
import time

objectif_atteint = False
max_iteration = 500
freq_mise_a_jour = 10

Magasin.InitImageTrajectoire()

iteration = 0
etat_courant = Magasin._ETAT_DEPART

display(widget_presentation)

time.sleep(1)

while objectif_atteint is False and iteration <= max_iteration:
    widget_etat.value = str(etat_courant)

    # Récupère la table des valeurs d'action de l'état en cours
    Q_ = Magasin.Q_table[etat_courant,:]

    # Récupère l'action optimale
    # si plusieurs actions sont optimales, alors tire une au hasard
    action = np.random.choice(np.where(Q_ == np.max(Q_))[0])

    # Simule l'action
    etat_courant, image_ = Magasin.SimuleAction(etat_courant,action)

    if etat_courant == Magasin.ETAT_CIBLE:
        objectif_atteint = True

    # Affiche la trajectoire
    if iteration%freq_mise_a_jour == 0:
      widget_image.value = bgr8_to_jpeg(image_)
      widget_iteration.value = str(iteration+1)

    iteration = iteration + 1
    time.sleep(0.1)